In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pathlib
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import sys
import time
import torchvision
import glob
from torch.autograd import Variable
from torch.optim import Adam
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from PIL import Image
from Model import Net

In [2]:
model_load = Net()
model_load.load_state_dict(torch.load('best_checkpoint.model'))

<All keys matched successfully>

In [3]:
model_load.eval()

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (relu1): ReLU()
  (maxpool1): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 12, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (relu2): ReLU()
  (maxpool2): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(12, 18, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (relu3): ReLU()
  (fc1): Linear(in_features=2592, out_features=120, bias=True)
  (dropout1): Dropout2d(p=0.05, inplace=False)
  (relufc1): ReLU()
  (fc2): Linear(in_features=120, out_features=80, bias=True)
  (relufc2): ReLU()
  (fc3): Linear(in_features=80, out_features=7, bias=True)
)

In [4]:
os.getcwd()

'D:\\Main Current PC\\Book\\DL Project'

In [5]:
def convert_image(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.resize(gray, (48, 48))
#     gray.resize(48, 48)
    gray1 = gray.copy()
    
    gray_np = np.array([[gray]])
#     gray = gray.resize((48, 48))
    final_image = Variable(torch.Tensor(gray_np))
    
    return final_image, gray1
    

In [8]:
image = cv2.imread('1.png')
image, _ = convert_image(image)
output = model_load(image)
output

tensor([[  37.1277, -377.9313,   37.9646,   22.0699,   94.2302,   44.9628,
         -103.8399]], grad_fn=<AddmmBackward0>)

In [43]:
moods = ['angry', 'disgusted', 'fearful', 'happy', 'neutral', 'sad', 'surprised']

In [36]:
output_list = output.detach().numpy()
moods[np.argmax(output_list[0])]

In [42]:
output_list

array([[  37.12772 , -377.93134 ,   37.964634,   22.069893,   94.23016 ,
          44.962788, -103.83987 ]], dtype=float32)

In [44]:
moods[np.argmax(output_list[0])]

'neutral'

In [20]:
output['indices']

TypeError: tuple indices must be integers or slices, not str

## Final Integration

In [9]:
cascPath = os.path.dirname(cv2.__file__) + "/data/haarcascade_frontalface_alt2.xml"
faceCascade = cv2.CascadeClassifier(cascPath)

In [10]:
video_capture = cv2.VideoCapture(0)
count = 0
while True:
    # Capture frame-by-frame
    ret, frame = video_capture.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(gray,
                                         scaleFactor=1.1,
                                         minNeighbors=5,
                                         minSize=(60, 60),
                                         flags=cv2.CASCADE_SCALE_IMAGE)
    face_detected = 0
    for (x,y,w,h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h),(0,255,0), 2)
        face_detected = 1
        # Display the resulting frame
    moods = ['happy','sad','angry','disgusted','neutral','surprised','fearful']
    
    path = 'D:\\Main Current PC\\Book\\emotion\\'
    
#     cv2.imshow('Video', frame)
    if face_detected == 1:
        roi = frame[y:y+h,x:x+w]
        cv2.imshow('Face', roi)
    key = cv2.waitKey(5) & 0xFF
    
    #Saving the image in their folder
#     if ord('p') == key and count == 0:
    image, grayimg = convert_image(roi)
    output = model_load(image)
    cv2.putText(frame,str(torch.max(output.data, 1)),(10, 120), cv2.FONT_HERSHEY_PLAIN, 1, (255,255,255), 1)
    cv2.imshow('Video', frame)
    cv2.imshow('Grayscale', grayimg)

#     print(str(torch.max(output.data, 1)))
    output = torch.max(output.data, 1)

    if ord('s') == key or count!=0:
        if count == 0:
            choice = int(input('''Choose From the following : 
            1. happy
            2. sad
            3. angry
            4. disgusted
            5. neutral
            6. surprised
            7. fearful'''))
            curr_path = path + moods[choice - 1]
            if os.getcwd() != curr_path:
                os.chdir(curr_path)
            print(os.getcwd())
#         if count == 0:
            totFiles = len([item for item in os.listdir(curr_path)])
            print("Total Images = ",totFiles)
#         if face_detected != 0:
#         if count == 0:
            currCount = 1 + totFiles
            count = currCount
        
        if face_detected == 1:
            print("Count:",count)
#             currCount = count
            img_name = "{}.png".format(count)

            cv2.imwrite(img_name, roi)
            print(f"Image Written {img_name}")
            count += 1
            time.sleep(0.5)
            if count >10:
                count = 0
                

    if ord('q') == key and count==0:
        break
video_capture.release()
cv2.destroyAllWindows()

error: OpenCV(4.5.1) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-kh7iq4w7\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
